# Benchmark network analysis

In [ ]:
import networkx as nx
import networkx.algorithms.community as nx_comm
from networkx.generators.community import LFR_benchmark_graph
from networkx.algorithms import bipartite
import numpy as np
import scipy as sp
from scipy import sparse
from cdlib import algorithms
from cdlib import evaluation
from utils import *
#from distances import *
#from consensus import *
import math

## Generate benchmark graphs

- https://arxiv.org/pdf/0805.4770.pdf
- https://networkx.org/documentation/stable/reference/generated/networkx.generators.community.LFR_benchmark_graph.html
- https://stackoverflow.com/questions/53608425/how-tune-lfr-benchmark-graph-method-in-networkx-for-generating-large-graph

In [ ]:
n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4, 5, 6, 7, 8, 9]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            G = LFR_benchmark_graph(n, (float(gamma) / 10), (float(beta) / 10), (float(mu) / 10), seed=10, min_degree=10, max_degree=50)
            m = nx.to_scipy_sparse_array(G)
            sp.io.mmwrite(fileprefix + fname + ".mtx", m)
            clust_lst = {frozenset(G.nodes[v]["community"]) for v in G}
            #nx.write_edgelist(G, fileprefix + name + ".edgelist", data=False)
            write_clust_lst(clust_lst, fileprefix + fname + ".gt")

In [ ]:
algs = ["infomap", "louvain", "leiden", "cnm", "label-prop", "markov", "walktrap", "spinglass"]
n = 1000
fileprefix = "LFR/" + "n" + str(n) + "/"
mus = [1, 2, 3, 4, 5, 6, 7, 8, 9]
gammas = [30]
betas = [11]
for mu in mus:
    for gamma in gammas:
        for beta in betas:
            fname = "LFR_n" + str(n) + "_mu0" + str(mu) + "_gamma" + str(gamma) + "_beta" + str(beta)
            graph_file = fileprefix + fname + ".mtx"
            G = None
            with open(graph_file) as f:
                G = nx.from_scipy_sparse_matrix(spio.mmread(f), create_using=nx.Graph)
                #print(len(G.nodes()), len(G.edges()))
                comms = None
                for alg in algs:
                    print("Running", alg, "on", graph_file)
                    if alg == "infomap":
                        coms = algorithms.infomap(G)
                    elif alg == "markov":
                        coms = algorithms.markov_clustering(G)
                    elif alg == "louvain":
                        coms = algorithms.louvain(G)
                    elif alg == "leiden":
                        coms = algorithms.leiden(G)
                    elif alg == "label-prop":
                        coms = algorithms.label_propagation(G)
                    elif alg == "cnm":
                        coms = algorithms.greedy_modularity(G)
                    elif alg == "walktrap":
                        coms = algorithms.walktrap(G)
                    elif alg == "spinglass":
                        coms = algorithms.spinglass(G)
                    print(mu, alg, len(coms.communities))
            #nx.write_edgelist(G, fileprefix + name + ".edgelist", data=False)
            #write_clust_lst(clust_lst, fileprefix + fname + ".gt")